In [1]:
#==============================================================================================
# sbert 모델을 각 4가지 평가 데이터셋으로 한꺼번에 평가하는 예시임
# => 평가 데이터 셋 : Korsts, Kluests, gluests, stsb_multi_mt 
# => 각 개별적 eval 출력 하는 예시는 sbert-test2.ipynb 참조
#=> ** skt/kobert-base-V1  테스트시에는  tokenizer_config.json 에 tokenizer_class:"XLNetTokenizer" 인지 확인해야 함
#==============================================================================================
import os
import sys
import time
from sentence_transformers import models, losses
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample

sys.path.append('../../')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert-test", logfilename="../../../log/s-bert-test")
device = GPU_info()
seed_everything(111)

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


logfilepath:../../../log/s-bert-test_2023-02-09.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
## 평가 설정 parameter 들 ########################################
# 평가할 bert 모델 혹은 sbert모델 경로
#model_path = "../../../data11/model/moco/cross/albert-small-kor-cross-sts4"
model_path = "bongsoo/kpf-cross-encoder-v1"

#===============================================================================
eval_batch_size = 64 #64 # 배치파일사이즈 = 크면 eval 시간이 단축됨.단 gpu 메모리 올라감.
max_seq_length = 128 #128
#===============================================================================

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = 'eval'
os.makedirs(output_path, exist_ok=True)
csvfilename = 'scratch'   # 유사도 저장할 파일명 (eval/similarity_evaluation_{csffilename}.csv 형식으로 저장)

# 평가 sts 형태의 말뭉치 파일들
use_korsts = 1     #  korsts 파일
use_kluests = 1    # kluests_v1.1 파일
use_glue_sts = 1   # true이면 영문 glue_sts 데이터셋 추가하여 평기 시킴
use_stsb = 1     # true이면 영문 stsb_multi_mt데이터셋 추가하여 평가시킴.)

# korsts tsv 파일 경로
korsts_file = '../../../data11/korpora/korsts/tune_test.tsv'

# kluests json 파일 경로
kluests_file = '../../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

# glue test tsv 파일경로 => validataion 데이터셋으로 eval 구하므로 주석처리함
#glue_test_file = '../../../data11/korpora/gluestsb/gluestsb-test.tsv'


In [3]:
model = CrossEncoder(model_path, max_length=max_seq_length, num_labels=1, device=device)
logger.info(f"*model path: {model_path}")
logger.info(f'{model}')

2023-02-09 16:39:24,459 - s-bert-test - INFO - *model path: bongsoo/kpf-cross-encoder-v1
2023-02-09 16:39:24,461 - s-bert-test - INFO - <sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x7f288b1cd5b0>


In [4]:
from typing import Dict, List, Optional

# 평가 함수 정의
def evalsts(model,                      # 모델
            output_path,                # eval 파일 출력 dir         
            eval_datasets: List[int],   # eval 데이터셋(리스트)
            csvfilename,                # eval 출력 파일 명 
           ):

    start = time.time()
    
    test_evaluator = CECorrelationEvaluator.from_input_examples(eval_datasets, name=csvfilename)
    result = test_evaluator(model, output_path=output_path)
   
    logger.info(f"\n")
    logger.info(f'=== result: {result} ===')
    logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
    logger.info("=====================================================")
    logger.info("\n")
    

In [5]:
# Korsts 평가 시작 
from datasets import load_dataset

if use_korsts == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*korsts file({korsts_file})")
    
    with open(korsts_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for line in lines:
            s1, s2, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts=[s1,s2], label=score))
            
    # 평가 시작
    evalsts(model, output_path, test_samples, csvfilename)
    

2023-02-09 16:39:24,836 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:24,837 - s-bert-test - INFO - *korsts file(../../../data11/korpora/korsts/tune_test.tsv)
2023-02-09 16:39:30,629 - s-bert-test - INFO - 

2023-02-09 16:39:30,631 - s-bert-test - INFO - === result: 0.8799168605203689 ===
2023-02-09 16:39:30,632 - s-bert-test - INFO - === 처리시간: 5.790 초 ===
2023-02-09 16:39:30,633 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:30,635 - s-bert-test - INFO - 



In [6]:
# Kluests 평가 시작 
from datasets import load_dataset
import json
    
if use_kluests == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*Kluests file({kluests_file})")
    
    with open(kluests_file, "r", encoding='utf-8') as fIn2:
        data = json.load(fIn2)
        for el in data:
            s1 = el["sentence1"]
            s2 = el["sentence2"]
            score = el["labels"]['label']
            test_samples.append(InputExample(texts=[s1,s2], label=score))
            
    # 평가 시작
    evalsts(model, output_path, test_samples, csvfilename)

2023-02-09 16:39:30,645 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:30,647 - s-bert-test - INFO - *Kluests file(../../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json)
2023-02-09 16:39:31,040 - s-bert-test - INFO - 

2023-02-09 16:39:31,042 - s-bert-test - INFO - === result: 0.913306252951734 ===
2023-02-09 16:39:31,043 - s-bert-test - INFO - === 처리시간: 0.386 초 ===
2023-02-09 16:39:31,043 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:31,044 - s-bert-test - INFO - 



In [7]:
# gluests 평가 시작 
from datasets import load_dataset
    
if use_glue_sts == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*gluests")
    
    #glue stsb valildation 데이터셋 불러옴(subset : "stsb" = 1,500)
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    '''
    # glue-test는 허깅페이스 데이터셋에는 labe(score)가 -1로 등록되어 있어서, 실제 glue 사이트에서 org파일을 받아서 불러옴
    # => 해당 sts-test.tsv 파일에서 몇가지 주석달린 문장들은 제거함(약 300개)
    with open(glue_test_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for idx, line in enumerate(lines):
            if idx == 0:
                continue
            _, _, _, _, score, text_a, text_b = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    '''
            
    # 평가 시작
    evalsts(model, output_path, test_samples, csvfilename)

2023-02-09 16:39:31,051 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:31,053 - s-bert-test - INFO - *gluests
Reusing dataset glue (/MOCOMSYS/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
2023-02-09 16:39:33,386 - s-bert-test - INFO - 

2023-02-09 16:39:33,387 - s-bert-test - INFO - === result: 0.8625694282544851 ===
2023-02-09 16:39:33,387 - s-bert-test - INFO - === 처리시간: 1.095 초 ===
2023-02-09 16:39:33,388 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:33,389 - s-bert-test - INFO - 



In [8]:
# stsb_multi_mt 평가 시작 
from datasets import load_dataset
    
if use_stsb == True:
    test_samples = []
    
    logger.info("=====================================================")
    logger.info(f"*stsb_multi_mt")
    
    stsb_dataset = load_dataset("stsb_multi_mt", name="en", split="test")
    for data in stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
            
    # 평가 시작
    evalsts(model, output_path, test_samples, csvfilename)

2023-02-09 16:39:33,396 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:33,397 - s-bert-test - INFO - *stsb_multi_mt
Reusing dataset stsb_multi_mt (/MOCOMSYS/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)
2023-02-09 16:39:35,566 - s-bert-test - INFO - 

2023-02-09 16:39:35,567 - s-bert-test - INFO - === result: 0.8027435489884991 ===
2023-02-09 16:39:35,568 - s-bert-test - INFO - === 처리시간: 0.955 초 ===
2023-02-09 16:39:35,568 - s-bert-test - INFO - =====================================================
2023-02-09 16:39:35,569 - s-bert-test - INFO - 



In [9]:
# crossencoder 문장쌍 테스트
from sentence_transformers import CrossEncoder
#model_path = "bongsoo/albert-small-kor-cross-encoder-v1"
model = CrossEncoder(model_path)
scores = model.predict([('오늘 날씨가 좋다', '오늘 등산을 한다'), ('오늘 날씨가 흐리다', '오늘 비가 내린다')])
print(scores)

KeyboardInterrupt: 